In [24]:
import autogluon

In [25]:
from autogluon.tabular import TabularDataset, TabularPredictor

In [26]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

dataset = TabularDataset('combined.csv')

dataset['Price'] = pd.to_numeric(dataset['Price'], errors='coerce')

dataset = dataset.dropna(subset=['Price'])  # Remove rows with missing prices

df_train, df_test = train_test_split(dataset, test_size=0.25)

Loaded data from: combined.csv | Columns = 14 / 14 | Rows = 15311 -> 15311


In [27]:
train_data = df_train
train_data.head()

,Type,Price,Bedrooms,Bathrooms,Area,Furnished,Level,Compound,Payment_Option,Delivery_Date,Delivery_Term,City,Ad_Date,Scraping_Date
1320,Apartment,830000.0,3,2,130.0,Unknown,Highest,Dar Misr,Cash,Ready to move,Finished,Badr City,47 minutes ago,2023-03-15
10870,Apartment,2300000.0,3,2,223.0,Unknown,7,Unknown,Cash,Ready to move,Finished,Sheraton,3 hours ago,2023-04-06
8735,Apartment,2100000.0,3,2.0,170.0,No,3,Unknown,Cash,Ready to move,Finished,Heliopolis,3 hours ago,2023-03-30
90,Apartment,3700080.0,3,2,168.0,No,3,Zizinia El Mostakbal,Installment,2026,Unknown,Mostakbal City,3 minutes ago,2023-03-13
4855,Penthouse,4000000.0,3,3,225.0,Unknown,Highest,Badya Palm Hills,Cash or Installment,Ready to move,Not Finished,Sheikh Zayed,1 hour ago,2023-03-21


In [28]:
test_data = df_test
test_data.head()

,Type,Price,Bedrooms,Bathrooms,Area,Furnished,Level,Compound,Payment_Option,Delivery_Date,Delivery_Term,City,Ad_Date,Scraping_Date
5296,Apartment,4600000.0,2,3,160.0,Unknown,3,Sky Condos Sodic,Cash,Ready to move,Semi Finished,New Cairo - El Tagamoa,3 hours ago,2023-03-21
3814,Apartment,1350000.0,2,1,140.0,Yes,Ground,Unknown,Cash,Ready to move,Finished,Hurghada,3 months ago,2023-03-19
10251,Apartment,810000.0,3,3,231.0,No,Ground,Taj City,Cash or Installment,soon,Not Finished,New Cairo - El Tagamoa,1 hour ago,2023-04-06
7837,Apartment,2115000.0,3,3.0,168.0,Unknown,2,Zizinia El Mostakbal,Cash or Installment,2024,Unknown,Mostakbal City,3 days ago,2023-03-30
8378,Apartment,3084000.0,2,1.0,122.0,No,1,Alex West,Cash or Installment,soon,Finished,Borg al-Arab,1 hour ago,2023-03-30


In [29]:
price_column = 'Price'
train_data[price_column].head()

1320      830000.0
10870    2300000.0
8735     2100000.0
90       3700080.0
4855     4000000.0
Name: Price, dtype: float64

In [30]:
predictor_price = TabularPredictor(label=price_column, path="agModels_predict_price_normal").fit(
    train_data, 
    time_limit=14400, 
    presets='best_quality', 
    ag_args_fit={'num_cpus': 16}
)

Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.8.10
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1-NixOS SMP PREEMPT_DYNAMIC Sat Jul 27 09:40:36 UTC 2024
CPU Count:          32
Memory Avail:       55.07 GB / 62.71 GB (87.8%)
Disk Space Avail:   838.91 GB / 1816.45 GB (46.2%)
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels` value. Copies of AutoGluon will be fit on subsets of the data. Then holdout validation data is used to dete

In [31]:
predictor_price.evaluate(test_data)

{'root_mean_squared_error': -1792711.0044360387,
 'mean_squared_error': -3213812745426.071,
 'mean_absolute_error': -879309.2251788554,
 'r2': 0.42954873454936593,
 'pearsonr': 0.6767576573951378,
 'median_absolute_error': -489106.5}

In [32]:
import pandas as pd
predictor_price.leaderboard(test_data)

leaderboard_df_reg_normal = predictor_price.leaderboard(test_data, extra_info=True)

print(leaderboard_df_reg_normal)

                          model    score_test     score_val  \
0            XGBoost_r89_BAG_L2 -1.489468e+06 -1.677123e+06   
1                XGBoost_BAG_L2 -1.490281e+06 -1.678786e+06   
2            XGBoost_r33_BAG_L2 -1.496468e+06 -1.674571e+06   
3            XGBoost_r31_BAG_L2 -1.496982e+06 -1.663306e+06   
4            XGBoost_r49_BAG_L2 -1.497157e+06 -1.681748e+06   
..                          ...           ...           ...   
179   NeuralNetTorch_r86_BAG_L1 -3.690236e+06 -3.848162e+06   
180  NeuralNetTorch_r121_BAG_L1 -3.690236e+06 -3.848162e+06   
181  NeuralNetTorch_r121_BAG_L2 -3.690236e+06 -3.848162e+06   
182   NeuralNetTorch_r31_BAG_L1 -3.693109e+06 -3.848155e+06   
183   NeuralNetTorch_r31_BAG_L2 -3.694717e+06 -3.848161e+06   

                 eval_metric  pred_time_test  pred_time_val     fit_time  \
0    root_mean_squared_error       22.008239      16.883224  6822.122664   
1    root_mean_squared_error       22.011047      16.879199  6825.297140   
2    root_mean_

In [33]:
from IPython.display import FileLink
leaderboard_df_reg_normal.to_csv('autogluon_leaderboard_regression_normal.csv', index=False)
FileLink('autogluon_leaderboard_regression_normal.csv')

/opt/nb/autogluon_leaderboard_regression_normal.csv

In [34]:
#predict score
evaluation_results = predictor_price.evaluate(test_data)

#to df
evaluation_df_reg_normal = pd.DataFrame([evaluation_results])

print(evaluation_df_reg_normal)

#to CSV
evaluation_df_reg_normal.to_csv('autogluon_regression_predictorScore_normal.csv', index=False)

FileLink('autogluon_regression_predictorScore_normal.csv')

   root_mean_squared_error  mean_squared_error  mean_absolute_error        r2  \
0            -1.792711e+06       -3.213813e+12       -879309.225179  0.429549   

   pearsonr  median_absolute_error  
0  0.676758              -489106.5  


/opt/nb/autogluon_regression_predictorScore_normal.csv